In [ ]:
!pip -q install openai-agents gradio

In [ ]:
import os
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

# Verify that the key is set
print(f"OpenAI API key set: {bool(openai_api_key)}")


OpenAI API key set: True


In [ ]:
from agents import Agent, Runner

agent = Agent(
    name="Basic Agent",
    instructions="You are a helpful assistant. Respond in all caps.",
    model="gpt-4o-mini"
)

result = await Runner.run(agent, "Hello! How are you?")
result.final_output

"HELLO! I'M DOING GREAT, THANK YOU! HOW ABOUT YOU?"

In [ ]:
joke_agent = Agent(
    name="Joke Agent",
    instructions="You are a joke teller. You are given a topic and you need to tell a joke about it.",
    model="gpt-4o-mini"
)

topic = "Probability"
result = await Runner.run(joke_agent, topic)
result.final_output

'Why did the statistician bring a ladder to the bar?\n\nBecause he heard the drinks were on the top shelf and the odds were in his favor!'

In [ ]:
language_agent = Agent(
    name="Language Agent",
    instructions="You are a language expert. You are given a joke and you need to rewrite it in a different language.",
)

joke_result = await Runner.run(joke_agent, topic)
translated_result = await Runner.run(language_agent, f"Translate this joke to Spanish: {joke_result.final_output}")
print(f"Original joke:\n{joke_result.final_output}\n")
print(f"Translated joke:\n{translated_result.final_output}")

Original joke:
Why did the probability teacher break up with the statistics teacher?

Because she couldn’t handle his confidence intervals!

Translated joke:
¿Por qué la profesora de probabilidad rompió con el profesor de estadística?

¡Porque no podía manejar sus intervalos de confianza!


#Structured Outputs

In [ ]:
from pydantic import BaseModel
from agents import Agent

class Recipe(BaseModel):
    title: str
    ingredients: list[str]
    cooking_time: int # in minutes
    servings: int

recipe_agent = Agent(
    name="Recipe Agent",
    instructions=("You are an agent for creating recipes. You will be given the name of a food and your job"
                  " is to output that as an actual detailed recipe. The cooking time should be in minutes."),
    output_type=Recipe
)

response = await Runner.run(recipe_agent, "Italian Sasuage with Spaghetti")
recipe = response.final_output
print(recipe)

title='Italian Sausage with Spaghetti' ingredients=['4 Italian sausage links (mild or spicy, according to preference)', '250g spaghetti', '2 tablespoons olive oil', '1 onion, finely chopped', '2 cloves garlic, minced', '1 can (400g) crushed tomatoes', '1 teaspoon dried oregano', '1/2 teaspoon red pepper flakes (optional)', 'Salt and pepper, to taste', 'Fresh basil leaves, for garnish', 'Grated Parmesan cheese, for serving'] cooking_time=35 servings=4


#Tools

### Websearch Tool

In [ ]:
from agents import WebSearchTool

news_agent = Agent(
    name="News Reporter",
    instructions="You are a news reporter. Your job is to find recent news articles on the US Politics",
    model="gpt-4o",
    tools=[WebSearchTool()]
)

result = await Runner.run(news_agent, "find news")
print(result.final_output)

Here are the latest developments in U.S. politics as of April 29, 2025:

**Congressional Leadership and Committee Changes**

- **House Oversight Committee Leadership**: Representative Alexandria Ocasio-Cortez (AOC) is considering a bid to become the ranking member of the House Oversight Committee. This follows Representative Gerry Connolly stepping back due to cancer treatment. AOC's potential leadership could influence Democratic strategies and oversight activities in the House. ([axios.com](https://www.axios.com/newsletters/axios-hill-leaders-e7067bb0-242f-11f0-a543-0f906d9b0f91?utm_source=openai))

- **Senate Republican Leadership**: Senate Majority Leader John Thune and Whip John Barrasso have highlighted record-low illegal border crossings under President Trump's second term, presenting it as a significant legislative achievement for the GOP. ([axios.com](https://www.axios.com/newsletters/axios-hill-leaders-e7067bb0-242f-11f0-a543-0f906d9b0f91?utm_source=openai))

- **Senate Democ

In [ ]:
mcdonalds_agent = Agent(name="McDonalds Assistant",
              instructions=f"You are a helpful server at McDonalds, respond to questions by using the search tool",
              model="gpt-4o",
              tools=[WebSearchTool()]
              )


In [ ]:
result = await Runner.run(mcdonalds_agent, "How much is BigMac?")

print(result.final_output)

As of 2024, the average price of a Big Mac in the United States is approximately $5.29, reflecting a 21% increase from $4.39 in 2019. ([cbsnews.com](https://www.cbsnews.com/news/mcdonalds-menu-price-hikes-fast-food/?utm_source=openai)) However, prices can vary by location due to factors such as local operating costs and economic conditions. For instance, in Hawaii, a Big Mac costs around $6.75, while in Mississippi, it's about $4.75. ([mymcdonaldsmenuprices.com](https://mymcdonaldsmenuprices.com/how-much-does-a-big-mac-cost/?utm_source=openai)) Additionally, some reports have highlighted exceptional cases, such as an $18 Big Mac meal in Connecticut, which McDonald's has stated is not representative of typical pricing. ([cbsnews.com](https://www.cbsnews.com/news/mcdonalds-menu-price-hikes-fast-food/?utm_source=openai))


## McDonald's Addresses Big Mac Pricing Concerns:
- [McDonald's says $18 Big Mac meal was an 'exception' and news reports overstated its price increases](https://apnews

In [ ]:
result

RunResult(input='How much is BigMac?', new_items=[ToolCallItem(agent=Agent(name='McDonalds Assistant', instructions='You are a helpful server at McDonalds, respond to questions by using the search tool', handoff_description=None, handoffs=[], model='gpt-4o', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[WebSearchTool(user_location=None, search_context_size='medium')], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseFunctionWebSearch(id='ws_6810782cee5481918975ecf8a5b8c8fd030e1b288d80267b', status='completed', type='web_search_call'), type='tool_call_item'), MessageOutputItem(agent=Agent(name='McD

In [ ]:
result.new_items[1].raw_item.content[0].annotations

[AnnotationURLCitation(end_index=235, start_index=134, title='Top McDonald\'s exec says $18 Big Mac meal is "exception," not the rule - CBS News', type='url_citation', url='https://www.cbsnews.com/news/mcdonalds-menu-price-hikes-fast-food/?utm_source=openai'),
 AnnotationURLCitation(end_index=550, start_index=438, title='2024 - How much does a Big Mac cost?', type='url_citation', url='https://mymcdonaldsmenuprices.com/how-much-does-a-big-mac-cost/?utm_source=openai'),
 AnnotationURLCitation(end_index=829, start_index=728, title='Top McDonald\'s exec says $18 Big Mac meal is "exception," not the rule - CBS News', type='url_citation', url='https://www.cbsnews.com/news/mcdonalds-menu-price-hikes-fast-food/?utm_source=openai'),
 AnnotationURLCitation(end_index=1064, start_index=884, title="McDonald's says $18 Big Mac meal was an 'exception' and news reports overstated its price increases", type='url_citation', url='https://apnews.com/article/c9c4abef25369f6b87b2781f5658bc0e?utm_source=open


# Custom function

In [ ]:
from agents import Agent, function_tool


@function_tool
def get_weather(city: str) -> str:
    """Fetch the weather for a given city.

    Args:
        city: The city to fetch the weather for.
    """
    print(f"Getting weather for {city}")
    return "sunny"

@function_tool
def get_temperature(city: str) -> str:
    """
    Get the temperature for a given city.
    """
    print(f"Getting temperature for {city}")
    return "70 degrees"

agent = Agent(
    name="Weather Agent",
    instructions="You are the local weather agent. You are given a city and you need to tell the weather or temperature, whatever was asked. It can be both. For any unrelated queries, say I cant help with that.",
    tools=[get_weather, get_temperature]
)

result = await Runner.run(agent, " Weather in Dallas")
result.final_output


Getting weather for Dallas


'The weather in Dallas is sunny.'

In [ ]:
result = await Runner.run(agent, " Temp in New Jersey")
result.final_output

Getting temperature for New Jersey


'The temperature in New Jersey is 70 degrees.'

In [ ]:
result = await Runner.run(agent, "Atlanta")
result.final_output

"I can't help with that."

# Agents as Tools

In [ ]:
nike_agent = Agent(
    name="Nike agent",
    instructions="You have information about Nike products",
)

ikea_agent = Agent(
    name="ikea agent",
    instructions="You have information about IKEA products",
)

orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are an allottment agent. You use the tools given to you to assign products to Nike or Ikea"
        "If they dont belong to either, say 'This doesn't belong to any of the brands'."
    ),
    tools=[
        nike_agent.as_tool(
            tool_name="assign_to_nike",
            tool_description="Assign Products to Nike",
        ),
        ikea_agent.as_tool(
            tool_name="assign_to_ikea",
            tool_description="Assign Products to Ikea",
        ),
    ],
)

result = await Runner.run(orchestrator_agent, input="Lipstick")
print(result.final_output)

This doesn't belong to any of the brands.


#Handoffs

In [ ]:
from agents import Agent, Runner, handoff, RunContextWrapper

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide assistance with math queries. Explain your reasoning at each step and include examples"
)

def on_math_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to math tutor agent")

def on_history_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to history tutor agent")

# This agent has the capability to handoff to either the history or math tutor agent
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question." +
    "If neither agent is relevant, provide a general response.",
    handoffs=[handoff(history_tutor_agent, on_handoff=on_history_handoff),
              handoff(math_tutor_agent, on_handoff=on_math_handoff)]
)
result = await Runner.run(triage_agent, "How do I add 2 and 2?")
result.final_output


Handing off to math tutor agent


"Adding two numbers is a basic arithmetic operation. Here's how you do it:\n\n1. **Identify the Numbers:** You're adding 2 and 2.\n\n2. **Understand the Operation:** Addition combines quantities. In this case, you are combining two groups of 2.\n\n3. **Perform the Addition:**\n   - Start with the first 2.\n   - Add the second 2.\n\n   \\[\n   2 + 2 = 4\n   \\]\n\n4. **Visualize (Optional):** If you think of objects, like apples:\n   - You have 2 apples.\n   - You add 2 more apples.\n   - You now have 4 apples in total.\n\nThat's it! The sum of 2 and 2 equals 4."

In [ ]:
result = await Runner.run(triage_agent, "How did WW2 start?")
result.final_output

Handing off to history tutor agent


'World War II began on September 1, 1939, when Germany, led by Adolf Hitler, invaded Poland. This aggressive expansion came after a period of increasing tension and militarization in Europe.\n\nHere’s a breakdown of the key events and context leading up to the war:\n\n1. **Treaty of Versailles (1919):**\n   - The treaty formally ended World War I but imposed heavy reparations and territorial losses on Germany. This created a sense of resentment and economic hardship.\n\n2. **Rise of Totalitarian Regimes:**\n   - In the 1930s, several aggressive totalitarian regimes emerged. Adolf Hitler became Chancellor of Germany in 1933, and his Nazi Party quickly established a dictatorship.\n   - Benito Mussolini had already established a fascist regime in Italy, while Japan was expanding its influence in Asia.\n\n3. **Expansionist Policies:**\n   - Hitler pursued aggressive expansion to create a greater German empire. He defied the Treaty of Versailles by rebuilding the military and annexing terri

In [ ]:
result = await Runner.run(triage_agent, "What is love?")
result.final_output

'Love is a complex and multifaceted emotion that can encompass a range of feelings, states, and attitudes. It often involves care, affection, and attachment towards another person, but it can also include passion, desire, commitment, and respect. Love can be felt for friends, family, romantic partners, or even activities, places, and ideals. Throughout history, philosophers, poets, and scientists have explored its meanings and implications, making it a subject of enduring curiosity and significance.'

#Agent Patterns


In [ ]:
from __future__ import annotations

import asyncio
from dataclasses import dataclass
from typing import Literal

from agents import Agent, ItemHelpers, Runner, TResponseInputItem,trace

"""
This example shows the LLM as a judge pattern. The first agent generates an outline for a story.
The second agent judges the outline and provides feedback. We loop until the judge is satisfied
with the outline.
"""

story_outline_generator = Agent(
    name="story_outline_generator",
    instructions=(
        "You generate a very short story outline based on the user's input."
        "If there is any feedback provided, use it to improve the outline."
    ),
    model="gpt-4o"
)


@dataclass
class EvaluationFeedback:
    feedback: str
    score: Literal["pass", "needs_improvement", "fail"]


evaluator = Agent[None](
    name="evaluator",
    instructions=(
        "You evaluate a story outline and decide if it's good enough."
        "If it's not good enough, you provide feedback on what needs to be improved."
        "Never give it a pass on the first try. Pass or fail it within 3 tries"
    ),
    model="gpt-4o",
    output_type=EvaluationFeedback,
)


async def main() -> None:
    msg = input("What kind of story would you like to hear? ")
    input_items: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

    latest_outline: str | None = None

    # We'll run the entire workflow in a single trace
    with trace("LLM as a judge"):
        while True:
            story_outline_result = await Runner.run(
                story_outline_generator,
                input_items,
            )

            input_items = story_outline_result.to_input_list()
            latest_outline = ItemHelpers.text_message_outputs(story_outline_result.new_items)
            print("Story outline generated")
            print(latest_outline)
            evaluator_result = await Runner.run(evaluator, input_items)
            result: EvaluationFeedback = evaluator_result.final_output

            print(f"Evaluator score: {result.score}")

            if result.score == "pass":
                print("Story outline is good enough, exiting.")
                break

            print("Re-running with feedback")

            input_items.append({"content": f"Feedback: {result.feedback}", "role": "user"})

    print(f"Final story outline: {latest_outline}")


if __name__ == "__main__":
    asyncio.run(main())

What kind of story would you like to hear? Teaching Assistant taking a data science course
Story outline generated
**Title: The Unlikely Learner**

**Outline:**

**Setting:**  
A modern university campus bustling with students and technology.

**Main Characters:**  
- **Alex Rivera:** A passionate Teaching Assistant (TA) with a background in literature.  
- **Dr. Emily Chen:** Renowned data science professor known for her innovative teaching methods.  
- **Sam Patel:** A curious computer science major and Alex’s study partner.

**Plot Summary:**

1. **Introduction:**  
   Alex Rivera, a literature TA, unexpectedly finds themselves enrolled in a cutting-edge data science course to keep up with academic requirements. Despite initial doubts, Alex is determined to bridge the gap between humanities and technology.

2. **Rising Action:**  
   Struggling with complex algorithms and data sets, Alex partners with Sam Patel. Together, they explore new tools and techniques, blending Alex’s analyt

In [ ]:
# prompt: async def main() -> None:
#     msg = input("What kind of story would you like to hear? ")
#     input_items: list[TResponseInputItem] = [{"content": msg, "role": "user"}]
#     latest_outline: str | None = None
#     # We'll run the entire workflow in a single trace
#     with trace("LLM as a judge"):
#         while True:
#             story_outline_result = await Runner.run(
#                 story_outline_generator,
#                 input_items,
#             )
#             input_items = story_outline_result.to_input_list()
#             latest_outline = ItemHelpers.text_message_outputs(story_outline_result.new_items)
#             print("Story outline generated")
#             evaluator_result = await Runner.run(evaluator, input_items)
#             result: EvaluationFeedback = evaluator_result.final_output
#             print(f"Evaluator score: {result.score}")
#             if result.score == "pass":
#                 print("Story outline is good enough, exiting.")
#                 break
#             print("Re-running with feedback")
#             input_items.append({"content": f"Feedback: {result.feedback}", "role": "user"})
#     print(f"Final story outline: {latest_outline}")
# if __name__ == "__main__":
#     asyncio.run(main())
# Change this code to suit the shopping planner agent above in such a loop like fashion

import os
from google.colab import userdata
from agents import Agent, Runner
from pydantic import BaseModel
from agents import Agent
from agents import Agent, function_tool
from agents import WebSearchTool
import asyncio
from agents import Agent, Runner, handoff, RunContextWrapper
from agents import function_tool
!pip -q install openai-agents gradio
openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

# Verify that the key is set
print(f"OpenAI API key set: {bool(openai_api_key)}")

# Shopping Agent
shopping_agent = Agent(
    name="Shopping Agent",
    instructions=(
        "You are a personal shopping assistant.  You will help the user find clothes "
        "within their specified budget. You have access to web search.  "
        "Prioritize finding items from reputable online retailers. Return a list of items "
        "with links and total price."
    ),
    tools=[WebSearchTool()],
    model="gpt-4o-mini" # Or a suitable model
)

# Budget Agent (Handoff)
budget_agent = Agent(
    name="Budget Agent",
    handoff_description="Handles budget adjustments and constraints.",
    instructions="You review the shopping cart and provide suggestions to stay within budget. "
                 "Suggest removing higher-priced items or finding cheaper alternatives.",
    model="gpt-4o-mini" # Or a suitable model
)


def on_budget_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to budget agent")


# Orchestrator Agent
orchestrator_agent = Agent(
    name="Shopping Orchestrator",
    instructions="You manage the shopping process.  Use the Shopping Agent to find items."
                "If the budget is exceeded, handoff to the Budget Agent for suggestions.",
    handoffs=[handoff(budget_agent, on_handoff=on_budget_handoff)],
    tools=[shopping_agent.as_tool(
        tool_name="find_clothes",
        tool_description="Search for clothes online within a given budget."
    )],
    model="gpt-4o-mini" # Or a suitable model
)

